In [1]:
pip install dash

     |████████████████████████████████| 7.3 MB 5.1 MB/s 
     |████████████████████████████████| 23.9 MB 1.3 MB/s 
     |████████████████████████████████| 357 kB 45.8 MB/s 
  Created wheel for dash-core-components: filename=dash_core_components-2.0.0-py3-none-any.whl size=3821 sha256=b17c9cc8e4f10fa2fd92d98174f96e6d44015951d60fc07374bf1d82c2a1611a
  Stored in directory: /root/.cache/pip/wheels/00/f9/c7/1a6437d794ed753ea9bc9079e761d4fc803a1f1f5d3697b9ec
  Created wheel for dash-html-components: filename=dash_html_components-2.0.0-py3-none-any.whl size=4089 sha256=fc26e5e63f304f3a2834a7b1315ff6dcbc96f16902e1297127c1ab5c7313d858
  Stored in directory: /root/.cache/pip/wheels/ec/6b/81/05aceabd8b27f724e2c96784016287cc1bfbc349ebfda451de
  Created wheel for dash-table: filename=dash_table-5.0.0-py3-none-any.whl size=3911 sha256=ecfb2df6abf055e47a982cecc584ba8a94b1aa12d865b7cc39f097ff2c4c5232
  Stored in directory: /root/.cache/pip/wheels/85/5d/4e/7c276b57992951dbe770bf5caad6448d0539c510663aef

In [2]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  after removing the cwd from sys.path.


In [3]:
path = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

In [5]:
spacex_df = pd.read_csv(path)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [6]:
spacex_df.drop(columns='Unnamed: 0', inplace=True)
spacex_df.head(2)

,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0


In [7]:
spacex_df['Launch Site'].value_counts()

CCAFS LC-40     26
KSC LC-39A      13
VAFB SLC-4E     10
CCAFS SLC-40     7
Name: Launch Site, dtype: int64

In [8]:
app = dash.Dash(__name__)

In [9]:
app.layout = html.Div(children=[
                                html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36','font-size': 40}),
                                dcc.Dropdown(id='site-dropdown', 
                                             options=[{'label':'All Sites', 'value':'All'},
                                                      {'label':'KSC LC-39A', 'value':'KSC LC-39A'}, 
                                                      {'label':'CCAFS LC-40', 'value':'CCAFS LC-40'},
                                                      {'label':'VAFB SLC-4E', 'value':'VAFB SLC-4E'},
                                                      {'label':'CCAFS SLC-40', 'value':'CCAFS SLC-40'}],
                                             value='All', searchable=True),
                                html.Br(),
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),
                                html.P('Payload range (Kg):'),
                                html.Div(dcc.RangeSlider(id='payload-slider', min=0, max=10000, step=1000, value=[min_payload, max_payload])),
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])



In [12]:
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):
  filtered_df = spacex_df
  if entered_site == 'ALL':
    fig = px.pie(filtered_df, values='class', names='Launch Site')
    return fig
  elif entered_site == 'KSC LC-39A':
    fig = px.pie(filtered_df[filtered_df['Launch Site']=='KSC LC-39A'], values='class', names='Launch Site')
    return fig
  elif entered_site == 'CCAFS LC-40':
    fig = px.pie(filtered_df[filtered_df['Launch Site']=='CCAFS LC-40'], values='class', names='Launch Site')
    return fig
  elif entered_site == 'VAFB SLC-4E':
    fig = px.pie(filtered_df[filtered_df['Launch Site']=='VAFB SLC-4E'], values='class', names='Launch Site')
    return fig
  else:
    fig = px.pie(filtered_df[filtered_df['Launch Site']=='CCAFS SLC-40'], values='class', names='Launch Site')
    return fig

In [13]:
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value'),
    Input(component_id='payload-slider', component_property='value')
)
def get_scatter_chart(entered_site, entered_range):
  range_list = format(entered_range)
  min = range_list[0]
  max = range_list[1]
  filtered_df = spacex_df[spacex_df['Payload Mass (kg)']<max and spacex_df['Payload Mass (kg)']>min]
  if entered_site == 'ALL':
    fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category')
    return fig
  elif entered_site == 'KSC LC-39A':
    fig = px.scatter(filtered_df[filtered_df['Launch Site']=='KSC LC-39A'], x='Payload Mass (kg)', y='class', color='Booster Version Category')
    return fig
  elif entered_site == 'CCAFS LC-40':
    fig = px.scatter(filtered_df[filtered_df['Launch Site']=='CCAFS LC-40'], x='Payload Mass (kg)', y='class', color='Booster Version Category')
    return fig
  elif entered_site == 'VAFB SLC-4E':
    fig = px.scatter(filtered_df[filtered_df['Launch Site']=='VAFB SLC-4E'], x='Payload Mass (kg)', y='class', color='Booster Version Category')
    return fig
  else:
    fig = px.scatter(filtered_df[filtered_df['Launch Site']=='CCAFS SLC-40'], x='Payload Mass (kg)', y='class', color='Booster Version Category')
    return fig

In [ ]:
if __name__ == '__main__':
    app.run_server()